In [1]:
import time
import matplotlib.pyplot as plt
# import seaborn as sns
# import statsmodels.api as sm
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from fbprophet import Prophet

In [2]:
import pymongo
from pymongo import MongoClient

mongo_uri = 'mongodb://localhost:27017/'

client = MongoClient(host=mongo_uri)
db = client.lasperrlive
payment_collections = db.reservations
reservations = payment_collections.find().sort(('date'))
reservations

In [3]:
reservations_df = pd.DataFrame(reservations)
reservations_df.head()

,_id,discountPercentage,waterAndElectricityInstallationFee,active,closed,year,plotId,estateId,legalFees,stampDuty,currencyId,contractSum,paymentPlanId,registrationFees,houseDesignTypeId,createdBy,totalAmountPayable,salesAgentId,clientId,rateId,initialDepositPercentage,whereDidYouHearAboutUsId,createdAt,__v,initialDeposit,numberOfMonths,updatedAt
0,5f071a9c673d820007aa8041,0,0,False,True,2020,5f0715548676eb0008154060,5f07154f8676eb0008153f14,17500,14250.00,5f07154f8676eb0008153f12,712500,5f07154f8676eb0008153ef5,1500,5f07154f8676eb0008153f09,5f0715a9a4a2880007cd5dca,745750,5f0715a9a4a2880007cd5dca,5f071a9c673d820007aa8040,5f07154f8676eb0008153f13,35.0,5f07154f8676eb0008153ee7,2020-07-09 13:24:44.389,0,249375,NaN,NaT
1,5f07292462cdc50007a83e5b,0,0,False,True,2020,5f0715558676eb00081540e2,5f07154f8676eb0008153f14,17500,13604.00,5f07154f8676eb0008153f12,680200,5f07154f8676eb0008153ef5,1500,5f07154f8676eb0008153f09,5f0715a9a4a2880007cd5dca,712804,5f0715a9a4a2880007cd5dca,5f07292462cdc50007a83e5a,5f07154f8676eb0008153f13,29.4,5f07154f8676eb0008153ee8,2020-07-09 14:26:44.300,0,199978,NaN,NaT
2,5f072ac96e417f0007c787bd,0,0,False,True,2020,5f0715548676eb000815404e,5f07154f8676eb0008153f14,17500,9500.00,5f07154f8676eb0008153f12,475000,5f07154f8676eb0008153ef5,1500,5f07154f8676eb0008153f09,5f072884130c670007855172,503500,5f072884130c670007855172,5f072ac96e417f0007c787bc,5f07154f8676eb0008153f13,20.0,5f07154f8676eb0008153eed,2020-07-09 14:33:45.835,0,95000,NaN,NaT
3,5f072c4011d8410007a75243,0,0,False,True,2020,5f0715558676eb0008154110,5f07154f8676eb0008153f14,17500,9500.00,5f07154f8676eb0008153f12,475000,5f07154f8676eb0008153ef5,1500,5f07154f8676eb0008153f09,5f0715a9a4a2880007cd5dca,503500,5f0715a9a4a2880007cd5dca,5f072c4011d8410007a75242,5f07154f8676eb0008153f13,42.1,5f07154f8676eb0008153eed,2020-07-09 14:40:00.848,0,199975,NaN,NaT
4,5f072ed821eb9f0008604dc3,0,0,False,True,2020,5f0715558676eb00081540dc,5f07154f8676eb0008153f14,291,158.32,5f07154f8676eb0008153f10,7916,5f07154f8676eb0008153ef5,25,5f07154f8676eb0008153f09,5f072b6a1e20fa00091325d0,8390,5f072b6a1e20fa00091325d0,5f072ed721eb9f0008604dc2,5f07154f8676eb0008153f11,20.0,5f07154f8676eb0008153ee7,2020-07-09 14:51:04.878,0,1583,NaN,NaT


In [4]:
reservation_cleaned = reservations_df[['year', 'closed', 'active', 'legalFees', 'stampDuty', 'contractSum', 'registrationFees', 'totalAmountPayable', 'initialDepositPercentage', 'initialDeposit']]
reservation_cleaned.head()

,year,closed,active,legalFees,stampDuty,contractSum,registrationFees,totalAmountPayable,initialDepositPercentage,initialDeposit
0,2020,True,False,17500,14250.00,712500,1500,745750,35.0,249375
1,2020,True,False,17500,13604.00,680200,1500,712804,29.4,199978
2,2020,True,False,17500,9500.00,475000,1500,503500,20.0,95000
3,2020,True,False,17500,9500.00,475000,1500,503500,42.1,199975
4,2020,True,False,291,158.32,7916,25,8390,20.0,1583


In [5]:
reservation_cleaned.closed.sum()

28

In [6]:
reservation_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      29 non-null     int64  
 1   closed                    29 non-null     bool   
 2   active                    29 non-null     bool   
 3   legalFees                 29 non-null     int64  
 4   stampDuty                 29 non-null     float64
 5   contractSum               29 non-null     int64  
 6   registrationFees          29 non-null     int64  
 7   totalAmountPayable        29 non-null     int64  
 8   initialDepositPercentage  29 non-null     float64
 9   initialDeposit            29 non-null     int64  
dtypes: bool(2), float64(2), int64(6)
memory usage: 2.0 KB


In [7]:
reservation_cleaned.describe()

,year,legalFees,stampDuty,contractSum,registrationFees,totalAmountPayable,initialDepositPercentage,initialDeposit
count,29.0,29.000000,29.000000,29.000000,29.000000,29.00000,29.000000,29.000000
mean,2020.0,11585.103448,7484.422069,376350.413793,992.931034,396412.62069,24.916552,110247.586207
std,0.0,8297.443941,5630.999105,283311.466386,711.318591,297532.00979,15.503421,106715.681965
min,2020.0,291.000000,158.320000,7916.000000,25.000000,8390.00000,0.090000,1583.000000
25%,2020.0,353.000000,230.300000,11515.000000,30.000000,12128.00000,20.000000,2303.000000
50%,2020.0,17500.000000,9500.000000,475000.000000,1500.000000,503500.00000,20.000000,99750.000000
75%,2020.0,17500.000000,11400.000000,570000.000000,1500.000000,600400.00000,21.000000,139840.000000
max,2020.0,17500.000000,15960.000000,798000.000000,1500.000000,832960.00000,85.890000,408000.000000


In [8]:
reservation_cleaned.closed.value_counts()

True     28
False     1
Name: closed, dtype: int64

In [9]:
reservation_cleaned.corr()

,year,closed,active,legalFees,stampDuty,contractSum,registrationFees,totalAmountPayable,initialDepositPercentage,initialDeposit
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
closed,NaN,1.000000,-1.000000,-0.137102,-0.133737,-0.131460,-0.137102,-0.131859,0.307985,-0.314946
active,NaN,-1.000000,1.000000,0.137102,0.133737,0.131460,0.137102,0.131859,-0.307985,0.314946
legalFees,NaN,-0.137102,0.137102,1.000000,0.953744,0.953363,1.000000,0.956126,0.230141,0.748085
stampDuty,NaN,-0.133737,0.133737,0.953744,1.000000,0.997781,0.953744,0.997897,0.156338,0.731605
contractSum,NaN,-0.131460,0.131460,0.953363,0.997781,1.000000,0.953363,0.999956,0.152202,0.728977
registrationFees,NaN,-0.137102,0.137102,1.000000,0.953744,0.953363,1.000000,0.956126,0.230141,0.748085
totalAmountPayable,NaN,-0.131859,0.131859,0.956126,0.997897,0.999956,0.956126,1.000000,0.154855,0.730633
initialDepositPercentage,NaN,0.307985,-0.307985,0.230141,0.156338,0.152202,0.230141,0.154855,1.000000,0.638028
initialDeposit,NaN,-0.314946,0.314946,0.748085,0.731605,0.728977,0.748085,0.730633,0.638028,1.000000


In [10]:
def connect_db():
    client = MongoClient(mongo_uri)
    db = client.lasperrlive
    return db

In [11]:
db = connect_db()
allocations_collection = db.allocations
allocations = allocations_collection.find()
allocations_df = pd.DataFrame(allocations)
allocations_df.head()

,_id,approved,year,plotId,clientId,estateId,stampDuty,legalFees,currencyId,contractSum,salesAgentId,paymentPlanId,initialDeposit,registrationFees,date,houseDesignTypeId,totalAmountPayable,discountPercentage,createdBy,initialDepositPercentage,waterAndElectricityInstallationFee,contractSumInWords,initialDepositInWords,createdAt,__v,active,closed,contractSumInGMD
0,5f0742ae7565300009f25b01,False,2020,5f0715558676eb0008154106,5f073e5623e8d6000890797b,5f07154f8676eb0008153f14,13300.00,17500,5f07154f8676eb0008153f12,631750,5f073bd72513d00008f9a86d,5f07154f8676eb0008153ef5,126350,1500,2020-07-09 16:15:42.765,5f07154f8676eb0008153f09,664050,5,5f073bd72513d00008f9a86d,20.0,0,"six hundred thirty-one thousand, seven hundred...","one hundred thirty-two thousand, eight hundred...",2020-07-09 16:15:42.768,0,True,True,631750
1,5f0743593d0d5b0007ece497,False,2020,5f0715558676eb000815414c,5f073ec7d36e7800087fd339,5f07154f8676eb0008153f14,10960.00,17500,5f07154f8676eb0008153f12,548000,5f072e8328399a0008963a68,5f07154f8676eb0008153ef5,109600,1500,2020-07-09 16:18:33.762,5f07154f8676eb0008153f09,577960,0,5f073bd72513d00008f9a86d,20.0,0,five hundred forty-eight thousand,"five hundred seventy-seven thousand, nine hund...",2020-07-09 16:18:33.765,0,True,True,548000
2,5f07439e1a4e1d000879a5d2,False,2020,5f0715548676eb000815408a,5f0733d3784c750008204636,5f07154f8676eb0008153f14,15960.00,17500,5f07154f8676eb0008153f12,798000,5f0715a9a4a2880007cd5dca,5f07154f8676eb0008153ef5,208240,1500,2020-07-09 16:19:42.182,5f07154f8676eb0008153f09,832960,0,5f073bd72513d00008f9a86d,25.0,0,seven hundred ninety-eight thousand,"two hundred eight thousand, two hundred forty",2020-07-09 16:19:42.186,0,True,True,798000
3,5f075962ca34a6000806cef9,False,2020,5f0715548676eb0008154042,5f0732db21eb9f0008604dc4,5f07154f8676eb0008153f14,276.36,353,5f07154f8676eb0008153f0c,13818,5f072b6a1e20fa00091325d0,5f07154f8676eb0008153ef5,2763,30,2020-07-09 17:52:34.485,5f07154f8676eb0008153f09,14477,0,5f073bd72513d00008f9a86d,20.0,0,"thirteen thousand, eight hundred eighteen","fourteen thousand, four hundred seventy-seven",2020-07-09 17:52:34.486,0,True,True,704718
4,5f0836d7bade5f0008f337c2,False,2020,5f0715558676eb0008154110,5f072c4011d8410007a75242,5f07154f8676eb0008153f14,9500.00,17500,5f07154f8676eb0008153f12,475000,5f0715a9a4a2880007cd5dca,5f07154f8676eb0008153ef5,211973,1500,2020-07-10 09:37:27.583,5f07154f8676eb0008153f09,503500,0,5f073bd72513d00008f9a86d,42.1,0,four hundred seventy-five thousand,"two hundred eleven thousand, nine hundred seve...",2020-07-10 09:37:27.587,0,True,True,475000


In [12]:
allocations_cleaned = allocations_df[['date', 'year', 'approved', 'stampDuty', 'legalFees', 'contractSum', 'initialDeposit', 'totalAmountPayable', 'discountPercentage', 'initialDepositPercentage', 'contractSumInWords', 'initialDepositInWords', 'closed', 'contractSumInGMD']]
allocations_cleaned.head()

,date,year,approved,stampDuty,legalFees,contractSum,initialDeposit,totalAmountPayable,discountPercentage,initialDepositPercentage,contractSumInWords,initialDepositInWords,closed,contractSumInGMD
0,2020-07-09 16:15:42.765,2020,False,13300.00,17500,631750,126350,664050,5,20.0,"six hundred thirty-one thousand, seven hundred...","one hundred thirty-two thousand, eight hundred...",True,631750
1,2020-07-09 16:18:33.762,2020,False,10960.00,17500,548000,109600,577960,0,20.0,five hundred forty-eight thousand,"five hundred seventy-seven thousand, nine hund...",True,548000
2,2020-07-09 16:19:42.182,2020,False,15960.00,17500,798000,208240,832960,0,25.0,seven hundred ninety-eight thousand,"two hundred eight thousand, two hundred forty",True,798000
3,2020-07-09 17:52:34.485,2020,False,276.36,353,13818,2763,14477,0,20.0,"thirteen thousand, eight hundred eighteen","fourteen thousand, four hundred seventy-seven",True,704718
4,2020-07-10 09:37:27.583,2020,False,9500.00,17500,475000,211973,503500,0,42.1,four hundred seventy-five thousand,"two hundred eleven thousand, nine hundred seve...",True,475000


In [13]:
allocations_cleaned.closed.value_counts(normalize=True) * 100

False    51.851852
True     48.148148
Name: closed, dtype: float64

In [14]:
allocations_cleaned.corr()

,year,approved,stampDuty,legalFees,contractSum,initialDeposit,totalAmountPayable,discountPercentage,initialDepositPercentage,closed,contractSumInGMD
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stampDuty,NaN,NaN,1.000000,0.949788,0.997600,0.725004,0.997723,0.202624,0.197510,0.109215,0.376002
legalFees,NaN,NaN,0.949788,1.000000,0.949403,0.738627,0.952383,0.138675,0.279722,0.052667,0.124188
contractSum,NaN,NaN,0.997600,0.949403,1.000000,0.722427,0.999953,0.176162,0.192140,0.091663,0.383327
initialDeposit,NaN,NaN,0.725004,0.738627,0.722427,1.000000,0.724065,0.032059,0.810792,0.149178,0.149314
totalAmountPayable,NaN,NaN,0.997723,0.952383,0.999953,0.724065,1.000000,0.175800,0.195176,0.090958,0.375957
discountPercentage,NaN,NaN,0.202624,0.138675,0.176162,0.032059,0.175800,1.000000,-0.078803,0.203519,0.136054
initialDepositPercentage,NaN,NaN,0.197510,0.279722,0.192140,0.810792,0.195176,-0.078803,1.000000,0.086234,-0.162412
closed,NaN,NaN,0.109215,0.052667,0.091663,0.149178,0.090958,0.203519,0.086234,1.000000,0.293798


In [16]:
db = connect_db()
plot_collections = db.plots
plots = plot_collections.find()
plots_df = pd.DataFrame(plots)
plots_df.head()

,_id,name,title,available,dimensions,index,blockId,estateId,plotTypeId,slug,plotPriceId,createdAt,__v
0,5f0715548676eb000815403e,TL92,TL92,True,250,4,5f07154f8676eb0008153f19,5f07154f8676eb0008153f14,5f07154e8676eb0008153ee0,tl92,5f0715508676eb0008153f1f,2020-07-09 13:02:12.195,0
1,5f0715548676eb000815403a,TL86,TL86,True,250,10,5f07154f8676eb0008153f19,5f07154f8676eb0008153f14,5f07154e8676eb0008153ee0,tl86,5f0715508676eb0008153f2d,2020-07-09 13:02:12.187,0
2,5f0715548676eb0008154040,TL94,TL94,True,250,2,5f07154f8676eb0008153f19,5f07154f8676eb0008153f14,5f07154e8676eb0008153ee0,tl94,5f0715508676eb0008153f23,2020-07-09 13:02:12.196,0
3,5f0715548676eb0008154042,TL95,TL95,True,360,1,5f07154f8676eb0008153f19,5f07154f8676eb0008153f14,5f07154e8676eb0008153ee0,tl95,5f07154f8676eb0008153f1d,2020-07-09 13:02:12.198,0
4,5f0715548676eb0008154046,TL116,TL116,True,255,2,5f07154f8676eb0008153f1a,5f07154f8676eb0008153f14,5f07154e8676eb0008153ee0,tl116,5f0715508676eb0008153f37,2020-07-09 13:02:12.215,0


In [19]:
plots_cleaned = plots_df[['_id', 'name', 'available', 'dimensions', 'index']]
plots_cleaned.head()

,_id,name,available,dimensions,index
0,5f0715548676eb000815403e,TL92,True,250,4
1,5f0715548676eb000815403a,TL86,True,250,10
2,5f0715548676eb0008154040,TL94,True,250,2
3,5f0715548676eb0008154042,TL95,True,360,1
4,5f0715548676eb0008154046,TL116,True,255,2


In [24]:
db = connect_db()
payment_plan_collections = db.paymentplans
payment_plan = payment_plan_collections.find()

In [25]:
payment_plan_df = pd.DataFrame(payment_plan)
payment_plan_df.head()

,_id,name,slug,createdAt,__v
0,5f07154f8676eb0008153ef3,Outright Buy,outright-buy,2020-07-09 13:02:07.289,0
1,5f07154f8676eb0008153ef5,18 Months,18-months,2020-07-09 13:02:07.289,0
2,5f07154f8676eb0008153ef4,Staged,staged,2020-07-09 13:02:07.289,0
3,5f07154f8676eb0008153ef2,Mortgage,mortgage,2020-07-09 13:02:07.288,0


In [26]:
payment_plan_cleaned = payment_plan_df[['_id', 'name']]
payment_plan_cleaned.head()

,_id,name
0,5f07154f8676eb0008153ef3,Outright Buy
1,5f07154f8676eb0008153ef5,18 Months
2,5f07154f8676eb0008153ef4,Staged
3,5f07154f8676eb0008153ef2,Mortgage


In [35]:
def get_collection_name(collection):
    db = connect_db()
    collection_name = db.collection
    collection_data = collection_name.find()
    return collection_data

In [42]:
db = connect_db()
estate_collection = db.estates
estates = estate_collection.find()

In [43]:
estates_df = pd.DataFrame(estates)
estates_df.head()

,_id,name,width,length,dimensions,slug,dateProcured,createdAt,__v,valueInGMD
0,5f07154f8676eb0008153f14,Tulip,90456,8765,90456 x 8765,tulip,2007-12-09,2020-07-09 13:02:07.528,0,74366800


In [44]:
estates_cleaned = estates_df[['_id', 'name', 'width', 'length', 'dimensions', 'dateProcured', 'valueInGMD']]
estates_cleaned

,_id,name,width,length,dimensions,dateProcured,valueInGMD
0,5f07154f8676eb0008153f14,Tulip,90456,8765,90456 x 8765,2007-12-09,74366800


In [49]:
db = connect_db()
clients_collection = db.clients
clients = clients_collection.find()

In [50]:
clients_df = pd.DataFrame(clients)
clients_df.head()

,_id,email,address,lastName,genderId,firstName,dateOfBirth,phoneNumber,nationalityId,createdBy,salesAgentId,countryOfResidenceId,__v,createdAt,updatedAt,updatedBy
0,5f071a9c673d820007aa8040,adamajeng1972@gmail.com,New Jeshwang,Jeng,5f07154d8676eb0008153e7e,Ebrima,1967-09-25T13:11:28.513Z,+2207424296,5f07154d8676eb0008153edc,5f0715a9a4a2880007cd5dca,5f0715a9a4a2880007cd5dca,5f07154d8676eb0008153e97,0,NaT,NaT,NaN
1,5f07292462cdc50007a83e5a,bintouhydaraceesay@gmail.com,"22 Cardinal Grove, Leeds, LS11 8HG ,UK",Ceesay/ Hydara Ceesay,5f07154d8676eb0008153e7f,Lamin / Bintou,1989-10-11T14:26:13.760Z,+447366120927,5f07154d8676eb0008153e97,5f0715a9a4a2880007cd5dca,5f0715a9a4a2880007cd5dca,5f07154d8676eb0008153edb,0,NaT,NaT,NaN
2,5f072ac96e417f0007c787bc,fatougibou@hotmail.com,Norway,Jagne Ngum,5f07154d8676eb0008153e7f,Fatou,1979-04-18T14:29:01.003Z,004748914767,5f07154d8676eb0008153e97,5f072884130c670007855172,5f072884130c670007855172,5f07154d8676eb0008153ec0,0,NaT,NaT,NaN
3,5f072c4011d8410007a75242,kingjammeh92@gmail.com,Lamin Village,Jammeh,5f07154d8676eb0008153e7e,Bakary,1992-07-17T14:33:25.339Z,+2207030232,5f07154d8676eb0008153e97,5f0715a9a4a2880007cd5dca,5f0715a9a4a2880007cd5dca,5f07154d8676eb0008153e97,0,NaT,NaT,NaN
4,5f072ed721eb9f0008604dc2,hydara198@gmail.com,6 Windrush Close Walsall W53 4 NW UK,Hydara,5f07154d8676eb0008153e7e,Abdoul Khadir,1988-12-09T08:00:00.000Z,+447916849231,5f07154d8676eb0008153e97,5f072b6a1e20fa00091325d0,5f072b6a1e20fa00091325d0,5f07154d8676eb0008153edb,0,NaT,NaT,NaN


In [51]:
clients_df_cleaned = clients_df[['_id', 'email', 'address', 'firstName', 'lastName', 'dateOfBirth', 'phoneNumber']]
clients_df_cleaned.head()

,_id,email,address,firstName,lastName,dateOfBirth,phoneNumber
0,5f071a9c673d820007aa8040,adamajeng1972@gmail.com,New Jeshwang,Ebrima,Jeng,1967-09-25T13:11:28.513Z,+2207424296
1,5f07292462cdc50007a83e5a,bintouhydaraceesay@gmail.com,"22 Cardinal Grove, Leeds, LS11 8HG ,UK",Lamin / Bintou,Ceesay/ Hydara Ceesay,1989-10-11T14:26:13.760Z,+447366120927
2,5f072ac96e417f0007c787bc,fatougibou@hotmail.com,Norway,Fatou,Jagne Ngum,1979-04-18T14:29:01.003Z,004748914767
3,5f072c4011d8410007a75242,kingjammeh92@gmail.com,Lamin Village,Bakary,Jammeh,1992-07-17T14:33:25.339Z,+2207030232
4,5f072ed721eb9f0008604dc2,hydara198@gmail.com,6 Windrush Close Walsall W53 4 NW UK,Abdoul Khadir,Hydara,1988-12-09T08:00:00.000Z,+447916849231


In [56]:
db = connect_db()
house_design_collection = db.housedesigntypes
house_design = house_design_collection.find()

In [57]:
house_design_df = pd.DataFrame(house_design)
house_design_df.head()

,_id,name,priceId,slug,createdAt,__v
0,5f07154f8676eb0008153eff,Maimuna 3 bed (100m2),5f07154f8676eb0008153efe,maimuna-3-bed-100m2,2020-07-09 13:02:07.303,0
1,5f07154f8676eb0008153ef7,Mariatou 2 bed (49m2),5f07154f8676eb0008153ef6,mariatou-2-bed-49m2,2020-07-09 13:02:07.301,0
2,5f07154f8676eb0008153efd,Majula 3 bed (90m2),5f07154f8676eb0008153efc,majula-3-bed-90m2,2020-07-09 13:02:07.302,0
3,5f07154f8676eb0008153efb,Mariatou 3 bed (70m2),5f07154f8676eb0008153efa,mariatou-3-bed-70m2,2020-07-09 13:02:07.302,0
4,5f07154f8676eb0008153ef9,Majula 2 bed (64m2),5f07154f8676eb0008153ef8,majula-2-bed-64m2,2020-07-09 13:02:07.302,0


In [59]:
house_design_df = house_design_df[['_id', 'name']]
house_design_df.head()

,_id,name
0,5f07154f8676eb0008153eff,Maimuna 3 bed (100m2)
1,5f07154f8676eb0008153ef7,Mariatou 2 bed (49m2)
2,5f07154f8676eb0008153efd,Majula 3 bed (90m2)
3,5f07154f8676eb0008153efb,Mariatou 3 bed (70m2)
4,5f07154f8676eb0008153ef9,Majula 2 bed (64m2)
